# WordPress 서버 클러스터 개선

## 개선점
- 도메인 연결(HTTPS)
- 모니터링 대시보드
    - CPU, 디스크 사용률
- CDN(CloudFront) - 블로그 같은 경우 변화가 많지 않기 떄문에 캐싱이 유리한 측면이 있음

## 데모 - 기존 아키텍처에 HTTPS 추가
- 클라우드포메이션으로 기존 아키텍처 되살리기
    - WordPress 셋업 전 까지
- ALB에 HTTPS 리스너 추가하여 HTTPS 구현
    - 사용자가 HTTP로 접속 시 HTTPS로 리다이렉션
    - 클라우드프론트 연동
- 클라우드워치 지표 기반 대시보드 생성하여 모니터링

### 실습 순서
1. 클라우드포메이션을 통해 아키텍처 되살리기
    - 서울리전 클라우드포메이션 콘솔로 이동
    - 스택 생성 : 템플릿 파일 업로드, yml 파일 선택
    - 도메인네임 : wordpress.lumifla.com
    - db패스워드 : abcd1234
    - cloudfront의 iam 리소스 생성 승인
2. ALB 수정
    - 리스너 추가
        - 프로토콜 : HTTPS
        - 대상 그룹 선택
        - 인증서 선택
3. Route53 레코드 생성, 접속 확인
    - wordpress.lumifla.com/wordpress로 워드프레스 설치해보자
    - 로그인
    - https://로 접속해보자.
    - http로 접속하면 https로 리다이렉션 되도록 만들려면?
        - ALB 리스너 수정
        - 80번 포트 기본 규칙 수정
            - HTTP로 받아서 URL로 리디렉션, 433번 포트 HTTPS 프로토콜로 받도록
        - 이러면 http로 접속해도 https로 리디렉션되는 것을 확인할 수 있다
4. 클라우드프론트 연동
    - 배포 생성
        - 오리진 : ALB
        - 프로토콜 : HTTP
        - 뷰어는 리다이렉트
        - 캐시 정책 disalbe
        - 원본 요청 정책 : all viewer
        - 보안 보호 비활성화
        - 대체 도메인 이름 : Wordpress.lumifla.com
        - 인증서 선택
5. ALB 리스너 규칙 수정
    - URL로 리디렉션을 대상 그룹으로 전달로 수정
6. Route53 레코드 수정(ALB -> CloudFront)
7. 로드밸런서 주소로 접근하는것 막기(클라우드프론트 우회 차단)
    - ec2 보안그룹 : 해당 보안그룹 인바운드 수정
        - 규칙 추가 : 모든 트래픽을 받되, 접두사 목록 중 cloudfront로부터만 받는다.
8. 클라우드워치 대시보드 만들기
    - ASG로 이동, asg 그룹 지표 수집 활성화
    - 클라우드워치 : 대시보드 생성
        - ASG 이름을 넣으면 그룹 지표 조회 가능
        - 네트워크 인아웃 그래프 생성
        - CPU 유틸리제이션 그래프 생성
        - 그룹 사이즈 관련 그래프 생성
9. 리소스 정리
    - 대시보드, 클라우드포메이션 스택

